In [202]:
# Author: Muhammad Fathur Rizky
# Date: 2024-12-24
# Email: rizkyfathur326@gmail.com

# **Requirements**

In [203]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

# **Data Preparation**

In [204]:
text = """
1 Singkawang 6,50 0,650 7,00 1,400 6,50 1,300 6,00 0,600 6,00 0,600 7,00 1,050 6,00 0,300 6,00 0,600 6,500
2 Bekasi 6,00 0,600 6,55 1,310 7,00 1,400 6,00 0,600 6,00 0,600 7,00 1,050 6,00 0,300 6,00 0,600 6,460
3 Salatiga 6,50 0,650 7,00 1,400 6,50 1,300 6,00 0,600 6,00 0,600 7,00 1,050 5,00 0,250 6,00 0,600 6,450
4 Manado 6,00 0,600 7,00 1,400 7,00 1,400 6,00 0,600 6,00 0,600 6,00 0,900 6,00 0,300 6,00 0,600 6,400
5 Semarang 6,00 0,600 6,90 1,380 6,50 1,300 6,00 0,600 6,00 0,600 6,00 0,900 5,00 0,250 6,00 0,600 6,230
6 Magelang 6,00 0,600 6,85 1,370 7,00 1,400 5,00 0,500 6,00 0,600 6,00 0,900 5,00 0,250 6,00 0,600 6,220
7 Kediri 5,83 0,583 6,70 1,340 7,00 1,400 5,00 0,500 6,00 0,600 6,00 0,900 5,00 0,250 5,00 0,500 6,073
8 Sukabumi 5,17 0,517 6,65 1,330 6,50 1,300 5,00 0,500 6,00 0,600 6,00 0,900 5,00 0,250 6,00 0,600 5,997
9 Kupang 5,33 0,533 6,85 1,370 7,00 1,400 6,00 0,600 5,00 0,500 5,00 0,750 6,00 0,300 5,00 0,500 5,953
10 Surakarta 5,50 0,550 7,00 1,400 6,00 1,200 5,00 0,500 5,00 0,500 5,00 0,750 6,00 0,300 6,00 0,600 5,800
11 Pematang Siantar 4,50 0,450 7,00 1,400 6,50 1,300 4,00 0,400 5,00 0,500 6,00 0,900 6,00 0,300 5,00 0,500 5,750
12 Bogor 4,50 0,450 5,40 1,080 6,50 1,300 6,00 0,600 6,00 0,600 7,00 1,050 5,00 0,250 4,00 0,400 5,730
13 Banjarmasin 4,83 0,483 5,65 1,130 6,50 1,300 6,00 0,600 6,00 0,600 6,00 0,900 4,00 0,200 4,00 0,400 5,613
14 Ambon 4,50 0,450 7,00 1,400 7,00 1,400 5,00 0,500 5,00 0,500 4,00 0,600 5,00 0,250 5,00 0,500 5,600
15 Bandung 5,17 0,517 6,65 1,330 6,00 1,200 5,00 0,500 5,00 0,500 5,00 0,750 5,00 0,250 5,00 0,500 5,547
16 Surabaya 4,83 0,483 5,55 1,110 5,50 1,100 6,00 0,600 5,00 0,500 6,00 0,900 5,00 0,250 6,00 0,600 5,543
17 Tomohon 5,00 0,500 7,00 1,400 7,00 1,400 3,00 0,300 4,00 0,400 5,00 0,750 4,00 0,200 5,00 0,500 5,450
18 Pontianak 4,00 0,400 7,00 1,400 7,00 1,400 5,00 0,500 5,00 0,500 4,00 0,600 5,00 0,250 4,00 0,400 5,450
19 Yogyakarta 3,83 0,383 7,00 1,400 7,00 1,400 4,00 0,400 4,00 0,400 6,00 0,900 3,00 0,150 4,00 0,400 5,433
20 Malang 4,50 0,450 5,60 1,120 6,00 1,200 5,00 0,500 6,00 0,600 5,00 0,750 5,00 0,250 5,00 0,500 5,370
21 Madiun 3,83 0,383 6,85 1,370 7,00 1,400 4,00 0,400 5,00 0,500 4,00 0,600 4,00 0,200 5,00 0,500 5,353
22 Batu 5,00 0,500 7,00 1,400 7,00 1,400 3,00 0,300 4,00 0,400 4,00 0,600 4,00 0,200 5,00 0,500 5,300
23 Jayapura 4,83 0,483 7,00 1,400 7,00 1,400 4,00 0,400 4,00 0,400 4,00 0,600 4,00 0,200 4,00 0,400 5,283
24 Probolinggo 4,67 0,467 7,00 1,400 7,00 1,400 4,00 0,400 5,00 0,500 4,00 0,600 2,00 0,100 4,00 0,400 5,267
25 Cimahi 4,67 0,467 6,70 1,340 7,00 1,400 3,00 0,300 4,00 0,400 5,00 0,750 4,00 0,200 4,00 0,400 5,257
26 Denpasar 4,33 0,433 7,00 1,400 6,00 1,200 3,00 0,300 4,00 0,400 5,00 0,750 5,00 0,250 5,00 0,500 5,233
27 Palu 4,17 0,417 7,00 1,400 7,00 1,400 3,00 0,300 4,00 0,400 5,00 0,750 3,00 0,150 4,00 0,400 5,217
28 Batam 4,00 0,400 6,70 1,340 7,00 1,400 3,00 0,300 4,00 0,400 4,00 0,600 5,00 0,250 5,00 0,500 5,190
29 Tegal 5,17 0,517 6,85 1,370 6,50 1,300 4,00 0,400 5,00 0,500 4,00 0,600 2,00 0,100 4,00 0,400 5,187
30 Sorong 4,33 0,433 7,00 1,400 7,00 1,400 4,00 0,400 4,00 0,400 3,00 0,450 4,00 0,200 5,00 0,500 5,183
31 Sibolga 4,33 0,433 7,00 1,400 7,00 1,400 3,00 0,300 4,00 0,400 5,00 0,750 4,00 0,200 3,00 0,300 5,183
32 Tanjung Pinang 5,83 0,583 6,50 1,300 7,00 1,400 4,00 0,400 4,00 0,400 4,00 0,600 4,00 0,200 3,00 0,300 5,183
33 Palangkaraya 4,50 0,450 6,85 1,370 6,50 1,300 4,00 0,400 4,00 0,400 4,00 0,600 5,00 0,250 4,00 0,400 5,170
34 Samarinda 3,50 0,350 6,35 1,270 6,50 1,300 4,00 0,400 5,00 0,500 5,00 0,750 4,00 0,200 4,00 0,400 5,170
35 Kendari 2,67 0,267 7,00 1,400 7,00 1,400 4,00 0,400 4,00 0,400 4,00 0,600 4,00 0,200 5,00 0,500 5,167
36 Bitung 4,50 0,450 7,00 1,400 7,00 1,400 4,00 0,400 4,00 0,400 3,00 0,450 5,00 0,250 4,00 0,400 5,150
37 Tangerang 3,50 0,350 6,45 1,290 6,50 1,300 4,00 0,400 5,00 0,500 5,00 0,750 3,00 0,150 4,00 0,400 5,140
38 Pangkal Pinang 4,33 0,433 7,00 1,400 7,00 1,400 3,00 0,300 4,00 0,400 4,00 0,600 4,00 0,200 4,00 0,400 5,133
39 Mojokerto 4,33 0,433 7,00 1,400 7,00 1,400 3,00 0,300 3,00 0,300 5,00 0,750 3,00 0,150 4,00 0,400 5,133
40 Tarakan 4,17 0,417 6,20 1,240 6,50 1,300 5,00 0,500 5,00 0,500 5,00 0,750 4,00 0,200 2,00 0,200 5,107
41 Tual 4,00 0,400 7,00 1,400 7,00 1,400 3,00 0,300 4,00 0,400 4,00 0,600 4,00 0,200 4,00 0,400 5,100
42 Pare Pare 4,00 0,400 7,00 1,400 6,50 1,300 4,00 0,400 4,00 0,400 5,00 0,750 3,00 0,150 3,00 0,300 5,100
43 Binjai 3,83 0,383 7,00 1,400 6,50 1,300 4,00 0,400 3,00 0,300 5,00 0,750 3,00 0,150 4,00 0,400 5,083
44 Kotamobagu 4,00 0,400 7,00 1,400 7,00 1,400 3,00 0,300 4,00 0,400 4,00 0,600 3,00 0,150 4,00 0,400 5,050
45 Gunung sitoli 3,83 0,383 6,50 1,300 6,50 1,300 4,00 0,400 5,00 0,500 5,00 0,750 4,00 0,200 2,00 0,200 5,033
46 Cirebon 5,33 0,533 6,70 1,340 6,00 1,200 4,00 0,400 3,00 0,300 3,00 0,450 4,00 0,200 6,00 0,600 5,023
47 Tangerang Selatan 4,00 0,400 6,60 1,320 6,50 1,300 4,00 0,400 4,00 0,400 5,00 0,750 3,00 0,150 3,00 0,300 5,020
48 Solok 4,50 0,450 6,55 1,310 7,00 1,400 4,00 0,400 4,00 0,400 4,00 0,600 3,00 0,150 3,00 0,300 5,010
49 Bengkulu 4,33 0,433 6,85 1,370 7,00 1,400 3,00 0,300 3,00 0,300 5,00 0,750 3,00 0,150 3,00 0,300 5,003
50 Sawahlunto 3,50 0,350 6,40 1,280 7,00 1,400 4,00 0,400 5,00 0,500 4,00 0,600 3,00 0,150 3,00 0,300 4,980
51 Langsa 2,83 0,283 6,40 1,280 7,00 1,400 4,00 0,400 4,00 0,400 4,00 0,600 4,00 0,200 4,00 0,400 4,963
52 Bukittinggi 4,00 0,400 5,95 1,190 7,00 1,400 5,00 0,500 4,00 0,400 4,00 0,600 3,00 0,150 3,00 0,300 4,940
53 Bontang 4,33 0,433 7,00 1,400 6,50 1,300 3,00 0,300 3,00 0,300 4,00 0,600 4,00 0,200 4,00 0,400 4,933
54 Gorontalo 4,00 0,400 6,90 1,380 6,50 1,300 2,00 0,200 3,00 0,300 5,00 0,750 4,00 0,200 4,00 0,400 4,930
55 Serang 4,00 0,400 6,15 1,230 7,00 1,400 4,00 0,400 4,00 0,400 4,00 0,600 2,00 0,100 4,00 0,400 4,930
56 Tanjung Balai 3,83 0,383 6,85 1,370 7,00 1,400 4,00 0,400 4,00 0,400 4,00 0,600 3,00 0,150 2,00 0,200 4,903
57 Banjar 3,83 0,383 6,80 1,360 7,00 1,400 4,00 0,400 5,00 0,500 2,00 0,300 3,00 0,150 4,00 0,400 4,893
58 Ternate 5,17 0,517 6,35 1,270 6,50 1,300 3,00 0,300 5,00 0,500 4,00 0,600 2,00 0,100 3,00 0,300 4,887
59 Tebing Tinggi 5,00 0,500 7,00 1,400 7,00 1,400 3,00 0,300 3,00 0,300 4,00 0,600 3,00 0,150 2,00 0,200 4,850
60 Jambi 4,50 0,450 6,90 1,380 6,00 1,200 4,00 0,400 4,00 0,400 4,00 0,600 4,00 0,200 2,00 0,200 4,830
61 Tasikmalaya 5,00 0,500 6,60 1,320 6,50 1,300 3,00 0,300 4,00 0,400 4,00 0,600 2,00 0,100 3,00 0,300 4,820
62 Makassar 5,17 0,517 4,60 0,920 4,50 0,900 5,00 0,500 5,00 0,500 5,00 0,750 4,00 0,200 5,00 0,500 4,787
63 Pasuruan 3,33 0,333 6,70 1,340 7,00 1,400 3,00 0,300 4,00 0,400 4,00 0,600 2,00 0,100 3,00 0,300 4,773
64 Palopo 3,33 0,333 7,00 1,400 6,50 1,300 3,00 0,300 3,00 0,300 3,00 0,450 3,00 0,150 5,00 0,500 4,733
65 Metro 3,00 0,300 7,00 1,400 7,00 1,400 3,00 0,300 3,00 0,300 3,00 0,450 3,00 0,150 4,00 0,400 4,700
66 Blitar 4,17 0,417 6,90 1,380 6,00 1,200 2,00 0,200 3,00 0,300 4,00 0,600 4,00 0,200 4,00 0,400 4,697
67 Lubuk Linggau 4,17 0,417 6,60 1,320 7,00 1,400 2,00 0,200 3,00 0,300 3,00 0,450 2,00 0,100 5,00 0,500 4,687
68 DKI Jakarta 5,50 0,550 6,40 1,280 3,50 0,700 4,00 0,400 4,00 0,400 4,00 0,600 5,00 0,250 5,00 0,500 4,680
69 Padang Panjang 3,17 0,317 6,70 1,340 7,00 1,400 3,00 0,300 4,00 0,400 3,00 0,450 3,00 0,150 3,00 0,300 4,657
70 Dumai 4,00 0,400 7,00 1,400 6,50 1,300 3,00 0,300 2,00 0,200 3,00 0,450 4,00 0,200 4,00 0,400 4,650
71 Prabumuli 2,00 0,200 6,55 1,310 7,00 1,400 3,00 0,300 5,00 0,500 3,00 0,450 3,00 0,150 3,00 0,300 4,610
72 Payakumbuh 3,67 0,367 6,45 1,290 6,00 1,200 3,00 0,300 4,00 0,400 4,00 0,600 3,00 0,150 3,00 0,300 4,607
73 Medan 4,17 0,417 6,10 1,220 3,50 0,700 5,00 0,500 4,00 0,400 5,00 0,750 4,00 0,200 4,00 0,400 4,587
74 Bima 3,83 0,383 7,00 1,400 7,00 1,400 2,00 0,200 2,00 0,200 3,00 0,450 3,00 0,150 4,00 0,400 4,583
75 Bau Bau 3,00 0,300 6,90 1,380 7,00 1,400 1,00 0,100 3,00 0,300 4,00 0,600 4,00 0,200 3,00 0,300 4,580
76 Banjarbaru 4,33 0,433 6,45 1,290 6,50 1,300 3,00 0,300 5,00 0,500 2,00 0,300 3,00 0,150 3,00 0,300 4,573
77 Tidore Kepulauan 4,00 0,400 6,85 1,370 6,50 1,300 3,00 0,300 3,00 0,300 3,00 0,450 3,00 0,150 3,00 0,300 4,570
78 Sungai Penuh 4,00 0,400 6,75 1,350 7,00 1,400 3,00 0,300 3,00 0,300 3,00 0,450 3,00 0,150 2,00 0,200 4,550
79 Pariaman 2,00 0,200 7,00 1,400 7,00 1,400 3,00 0,300 4,00 0,400 3,00 0,450 2,00 0,100 3,00 0,300 4,550
80 Balikpapan 3,67 0,367 6,60 1,320 7,00 1,400 4,00 0,400 4,00 0,400 2,00 0,300 3,00 0,150 2,00 0,200 4,537
81 Pagar Alam 3,33 0,333 7,00 1,400 7,00 1,400 3,00 0,300 2,00 0,200 3,00 0,450 3,00 0,150 3,00 0,300 4,533
82 Padangsidimpuan 3,00 0,300 7,00 1,400 7,00 1,400 2,00 0,200 3,00 0,300 3,00 0,450 3,00 0,150 3,00 0,300 4,500
83 Subulussalam 3,00 0,300 6,70 1,340 7,00 1,400 3,00 0,300 4,00 0,400 3,00 0,450 2,00 0,100 2,00 0,200 4,490
84 Pekalongan 3,17 0,317 7,00 1,400 4,00 0,800 3,00 0,300 5,00 0,500 4,00 0,600 3,00 0,150 4,00 0,400 4,467
85 Sabang 3,67 0,367 6,70 1,340 7,00 1,400 2,00 0,200 3,00 0,300 3,00 0,450 2,00 0,100 3,00 0,300 4,457
86 Bandar Lampung 3,00 0,300 6,25 1,250 5,00 1,000 3,00 0,300 4,00 0,400 4,00 0,600 4,00 0,200 4,00 0,400 4,450
87 Palembang 3,33 0,333 6,75 1,350 5,50 1,100 3,00 0,300 3,00 0,300 3,00 0,450 4,00 0,200 4,00 0,400 4,433
88 Pekanbaru 3,50 0,350 6,35 1,270 5,00 1,000 3,00 0,300 4,00 0,400 4,00 0,600 4,00 0,200 3,00 0,300 4,420
89 Mataram 3,67 0,367 5,85 1,170 5,50 1,100 3,00 0,300 4,00 0,400 4,00 0,600 3,00 0,150 3,00 0,300 4,387
90 Lhokseumawe 3,67 0,367 5,55 1,110 7,00 1,400 3,00 0,300 3,00 0,300 3,00 0,450 3,00 0,150 3,00 0,300 4,377
91 Padang 3,67 0,367 4,90 0,980 5,00 1,000 5,00 0,500 4,00 0,400 3,00 0,450 4,00 0,200 4,00 0,400 4,297
92 Banda Aceh 3,50 0,350 5,80 1,160 7,00 1,400 3,00 0,300 2,00 0,200 3,00 0,450 2,00 0,100 3,00 0,300 4,260
93 Cilegon 3,33 0,333 5,30 1,060 4,00 0,800 4,00 0,400 4,00 0,400 4,00 0,600 4,00 0,200 4,00 0,400 4,193
94 Depok 4,00 0,400 4,55 0,910 3,50 0,700 4,00 0,400 4,00 0,400 4,00 0,600 4,00 0,200 4,00 0,400 4,010
"""

cleaned_text = ""

In [205]:
for t in text.strip().split("\n"):
    txt = t
    txt = txt.strip().split()

    # merge the city name
    if txt[2].isalpha(): 
        txt[1] = txt[1] + " " + txt[2]
        txt.pop(2)
    # remove the comma
    txt = [i.replace(",", ".") for i in txt]
    # print(txt)
    # remove the last element

    cleaned_text += ";".join(txt) + "\n"

In [206]:
# to dataframe
data = pd.DataFrame([i.split(";") for i in cleaned_text.strip().split("\n")])
col = ['No', 'Kota', 'IND1', 'Weight1', 'IND2', 'Weight2', 'IND3', 'Weight3', 'IND4', 'Weight4', 'IND5', 'Weight5', 'IND6', 'Weight6', 'IND7', 'Weight7', 'IND8', 'Weight8', 'Skor Akhir']
data.columns = col
data = data.set_index('No')
for i in data.columns:
    if i != 'Kota':
        data[i] = data[i].astype(float)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94 entries, 1 to 94
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Kota        94 non-null     object 
 1   IND1        94 non-null     float64
 2   Weight1     94 non-null     float64
 3   IND2        94 non-null     float64
 4   Weight2     94 non-null     float64
 5   IND3        94 non-null     float64
 6   Weight3     94 non-null     float64
 7   IND4        94 non-null     float64
 8   Weight4     94 non-null     float64
 9   IND5        94 non-null     float64
 10  Weight5     94 non-null     float64
 11  IND6        94 non-null     float64
 12  Weight6     94 non-null     float64
 13  IND7        94 non-null     float64
 14  Weight7     94 non-null     float64
 15  IND8        94 non-null     float64
 16  Weight8     94 non-null     float64
 17  Skor Akhir  94 non-null     float64
dtypes: float64(17), object(1)
memory usage: 14.0+ KB


In [207]:
data.to_csv('Indeks Tolerasi Kota.csv', index=False)

In [208]:
gpd_data = gpd.read_file("indonesia-geo.json", index_col='ID')
gpd_data.set_index('ID', inplace=True)
gpd_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 32 entries, 1 to 32
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   kode             32 non-null     int32   
 1   Propinsi         32 non-null     object  
 2   SUMBER           32 non-null     object  
 3   Jumlah Penduduk  32 non-null     int32   
 4   geometry         32 non-null     geometry
dtypes: geometry(1), int32(2), object(2)
memory usage: 1.1+ KB


In [209]:
city_province = pd.read_csv("city_province.csv")
city_province.head()

,City,Province
0,Singkawang,KALIMANTAN BARAT
1,Bekasi,JAWA BARAT
2,Salatiga,JAWA TENGAH
3,Manado,SULAWESI UTARA
4,Semarang,JAWA TENGAH


In [210]:
# left join from data and gpd_data

data = data.merge(city_province, left_on='Kota', right_on='City', how='left')
data.drop(columns=['City'], inplace=True)
data = data.merge(gpd_data, left_on='Province', right_on='Propinsi', how='left')
data.drop(columns=['Propinsi', 'kode', 'SUMBER', 'Jumlah Penduduk'], inplace=True)
data.rename(columns={'Province': 'Provinsi'}, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Kota        94 non-null     object  
 1   IND1        94 non-null     float64 
 2   Weight1     94 non-null     float64 
 3   IND2        94 non-null     float64 
 4   Weight2     94 non-null     float64 
 5   IND3        94 non-null     float64 
 6   Weight3     94 non-null     float64 
 7   IND4        94 non-null     float64 
 8   Weight4     94 non-null     float64 
 9   IND5        94 non-null     float64 
 10  Weight5     94 non-null     float64 
 11  IND6        94 non-null     float64 
 12  Weight6     94 non-null     float64 
 13  IND7        94 non-null     float64 
 14  Weight7     94 non-null     float64 
 15  IND8        94 non-null     float64 
 16  Weight8     94 non-null     float64 
 17  Skor Akhir  94 non-null     float64 
 18  Provinsi    94 non-null     object  
 19  geometry  

# **Exploratory Data Analysis**